In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
os.environ['LLAMA_INDEX_TOKEN'] = os.getenv('LLAMA_INDEX_TOKEN')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')


TypeError: str expected, not NoneType

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_parse import LlamaParse
llama_cloud_api_key = os.environ['LLAMA_INDEX_TOKEN']
print(llama_cloud_api_key)

llx-u9CgBTDcgirzPcI6v5khnRFR01MdhAmPBn4P9gM1i1C73bLt


In [ ]:
documents = LlamaParse(
    api_key=llama_cloud_api_key,
    result_type="markdown",
    content_guideline_instruction="This is a resume, gather relevant facts together and format them as bullet points with headers",
).load_data(
    "data/resume.pdf",
)

Started parsing the file under job_id 96a416ea-e201-466c-a7c1-2ca4345faac3


In [ ]:
for document in documents:
    print(document.text)

# Aman Lonare

Email: amanlonare95@gmail.com

Phone: +81‑70‑9018‑5707

Location: Tokyo, Japan

# PROFESSIONAL EXPERIENCE

# Hitachi R&D, Center for Technology Innovation

Tokyo, JPN

Research Software Engineering, Services Computing Research Dept.
Jan 2021 – Present

- Developed a tool in Python for assisting in the implementation of CQRS and Event Sourcing design patterns for business users
- Designed and performed evaluation of Core Banking application using AxonIQ Framework with Prometheus and Grafana
- Reduced the overall development time by 15% using Domain Driven Design (DDD) principles of software development process
- Facilitated the implementation of event driven architecture in system and integrated Machine Learning solutions with the tool
- Proposed a plan to integrate the developed tool in Hitachi’s indigenous framework for microservices application development

Patent: “System and method to assist the modelling of CQRS and Event Sourcing based application”, In preparation


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [ ]:
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=Settings.embed_model,
)

In [ ]:
from llama_index.llms.groq import Groq

In [ ]:
groq_api_key = os.environ['GROQ_API_KEY']
llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)
llm

Groq(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x310575110>, system_prompt=None, messages_to_prompt=<function messages_to_prompt at 0x107a94ea0>, completion_to_prompt=<function default_completion_to_prompt at 0x107f172e0>, output_parser=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>, query_wrapper_prompt=None, model='llama-3.2-3b-preview', temperature=0.1, max_tokens=None, logprobs=None, top_logprobs=0, additional_kwargs={}, max_retries=3, timeout=60.0, default_headers=None, reuse_client=True, api_key='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', api_base='https://api.groq.com/openai/v1', api_version='', strict=False, reasoning_effort=None, modalities=None, audio_config=None, context_window=3900, is_chat_model=True, is_function_calling_model=True, tokenizer=None)

In [ ]:
query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=5,
)
response = query_engine.query(
    "What is this person name and what was their most recent job"
)


In [ ]:
storage_dir = "./storage"
index.storage_context.persist(
    persist_dir=storage_dir,
)

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage

if os.path.exists(storage_dir):
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    restored_index = load_index_from_storage(storage_context)
else:
    print("Index not found on disk")

In [ ]:
response = restored_index.as_query_engine(llm=llm, similarity_top_k=5,).query(
    "What is this person name and what was their most recent job"
)
print(response)

The person's name is Aman Lonare. Their most recent job is Research Software Engineering at Hitachi R&D, Center for Technology Innovation, where they have been working since January 2021.


In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

## The below function will be provided to the llm which will assess the docstring, name and metadata of the function and decides if it useful or not

In [ ]:
def query_resume(q: str):
    """Answer questions about a specific resume."""
    # we are using query engine we created from above
    response = query_engine.query(f"This is a question about the specific resume: {q}")
    return response.response


## We will turn the method into a tool using FunctionTool

In [ ]:

resume_tool = FunctionTool.from_defaults(
    fn=query_resume,
)

## We will create a function calling agent now

In [ ]:
agent = FunctionCallingAgent.from_tools(
    tools=[resume_tool], # just one in our case
    llm=llm,
    verbose=True,
)

In [ ]:
response = agent.chat("How many years of experience does this person have? Give me exact number of years as output")
print(response)

> Running step e3944d25-a673-448c-9c7b-d3a7dd6809e9. Step input: How many years of experience does this person have? Give me exact number of years as output
Added user message to memory: How many years of experience does this person have? Give me exact number of years as output
=== Calling Function ===
Calling function: query_resume with args: {"q": "years of experience"}
=== Function Output ===
The individual has around 7 years of experience in the field of software engineering and technology development.
> Running step 7d527efa-7988-4a0f-a12d-85b8789a4965. Step input: None
=== Calling Function ===
Calling function: query_resume with args: {"q": "number of years of experience"}
=== Function Output ===
Based on the provided context information, Aman Lonare has approximately 3 years of experience in the field of Research Software Engineering at Hitachi R&D, Center for Technology Innovation.
> Running step 8ca46822-d39b-48ea-bea2-b7ad1466ca5e. Step input: None
=== Calling Function ===
Ca

In [ ]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)

In [ ]:
class QueryEvent(Event):
    """Custom event to query the resume."""
    query: str

## RAG Workflow

In [ ]:
class RAGWorkflow(Workflow):
    storage_dir = "./storage"
    llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)
    query_engine = VectorStoreIndex

    # first step will be setup
    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> QueryEvent:
        
        if not ev.resume_file:
            raise ValueError("Resume file not provided")
        
        # define an llm to work with
        self.llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)

        # ingest data and set up the query engine
        if os.path.exists(self.storage_dir):
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse the data if storage index is not created
            documents = LlamaParse(
                api_key=llama_cloud_api_key,
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather relevant facts together and format them as bullet points with headers",
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=Settings.embed_model,
            )
            # persist the index
            index.storage_context.persist(
                persist_dir=self.storage_dir,
            )
        
        # either way, create query engine
        self.query_engine = index.as_query_engine(
            llm=self.llm,
            similarity_top_k=5,
        )

        # fire off the query event defined
        return QueryEvent(query=ev.query)
    
    # second step will be to ask a question and return a result immediately
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        # query the resume
        response = self.query_engine.query(f"This is the question about the specific resume: {ev.query}")
        # return the result
        return StopEvent(result=response.response)

In [ ]:
workflow = RAGWorkflow(timeout=60, verbose=False)
result = await workflow.run(
    resume_file="data/resume.pdf",
    query="Where is the first place the person worked?",
)
print(result)

Hitachi R&D, Center for Technology Innovation in Tokyo, Japan.


## Form Parsing

In [ ]:
parser = LlamaParse(
    api_key=llama_cloud_api_key,
    content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
    formatting_instruction="Return a bulleted list of the fields ONLY"
)

In [ ]:
result = parser.load_data(
    file_path="data/application_form.pdf",
)[0]

Started parsing the file under job_id eb53bb03-7362-4c53-817d-53e55e28eba2


In [ ]:
print(result.text)

First Name:
Last Name:
Email:
Phone:
LinkedIn:
Project Portfolio:
Degree:
Done Post Graduation?:
Post Graduation Date:
Graduation Date:
Current Job Title:
Current Employer:
Technical Skills:
Describe why the candidate is a good fit for this position:
Do you have 5 years of experience in Python?:


In [ ]:
llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)
raw_json = llm.complete(
    f"""
    This is a parsed from a job application form.
    Convert it into a JSON object containing only the list of fields
    to be filled in, in the form {{"fields": [...]}}.
    <form>{result.text}</form>
    Return JSON only, no markdown please. 
    Clean the text by removing '\n' or new line if there are any.
    Do not add the type and just return the field name as a list
    """
)

In [ ]:
raw_json

CompletionResponse(text='{"fields": ["First Name", "Last Name", "Email", "Phone", "LinkedIn", "Project Portfolio", "Degree", "Done Post Graduation?", "Post Graduation Date", "Graduation Date", "Current Job Title", "Current Employer", "Technical Skills", "Describe why the candidate is a good fit for this position", "Do you have 5 years of experience in Python?"]}', additional_kwargs={'prompt_tokens': 190, 'completion_tokens': 84, 'total_tokens': 274}, raw=ChatCompletion(id='chatcmpl-ea8a58c1-ed0d-49fc-8fea-7c0c217063d3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"fields": ["First Name", "Last Name", "Email", "Phone", "LinkedIn", "Project Portfolio", "Degree", "Done Post Graduation?", "Post Graduation Date", "Graduation Date", "Current Job Title", "Current Employer", "Technical Skills", "Describe why the candidate is a good fit for this position", "Do you have 5 years of experience in Python?"]}', refusal=None, role='assistant',

In [ ]:
import json
fields = json.loads(raw_json.text)['fields']

for field in fields:
    print(field)


First Name
Last Name
Email
Phone
LinkedIn
Project Portfolio
Degree
Done Post Graduation?
Post Graduation Date
Graduation Date
Current Job Title
Current Employer
Technical Skills
Describe why the candidate is a good fit for this position
Do you have 5 years of experience in Python?


In [ ]:
class ParseFormEvent(Event):
    """Custom event to parse the form."""
    application_form: str

class QueryEvent(Event):
    """Custom event to query the resume."""
    query: str

class ResponseEvent(Event):
    """Custom event to return the response."""
    response: str

## RAG Workflow

In [ ]:
class FormParserRAGWorkflow(Workflow):
    storage_dir = "./storage"
    llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)
    query_engine = VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")
        
        # define an llm to work with
        self.llm = Groq(model="llama-3.2-3b-preview", api_key=groq_api_key)

        # ingest data and set up the query engine
        if os.path.exists(self.storage_dir):
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse the data if storage index is not created
            parser = LlamaParse(
                api_key=llama_cloud_api_key,
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather relevant facts together and format them as bullet points with headers",
            ).load_data(ev.resume_file)
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=Settings.embed_model,
            )
            # persist the index
            index.storage_context.persist(
                persist_dir=self.storage_dir,
            )
        
        # either way, create query engine
        self.query_engine = index.as_query_engine(
            llm=self.llm,
            similarity_top_k=5,
        )

        # fire off the query event defined
        return ParseFormEvent(application_form=ev.application_form)
    
    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> QueryEvent:
        
        parser = LlamaParse(
        api_key=llama_cloud_api_key,
        content_guideline_instruction="This is a job application form. Create a list of all the fields that need to be filled in.",
        formatting_instruction="Return a bulleted list of the fields ONLY"
        )

        result = parser.load_data(
            file_path="data/application_form.pdf",
        )[0]

        raw_json = self.llm.complete(
            f"""
            This is a parsed from a job application form.
            Convert it into a JSON object containing only the list of fields
            to be filled in, in the form {{"fields": [...]}}.
            <form>{result.text}</form>
            Return JSON only, no markdown please. 
            Clean the text by removing '\n' or new line if there are any.
            Do not add the type and just return the field name as a list
            """
        )

        fields = json.loads(raw_json.text)['fields']

        for field in fields:
            ctx.send_event(
                QueryEvent(
                    field=field,
                    query=f"How would you answer the question about the candidate? Give me a succinct answer for each field if there is no description like first name etc.",
                    )
            )
    
        await ctx.set("total fields", len(fields))
        return

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        # query the resume
        response = self.query_engine.query(f"This is the question about the specific resume: {ev.query}")
        # return the result
        return ResponseEvent(field=ev.field, response=response.response)
    
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> StopEvent:
        total_fields = await ctx.get("total fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)

        if responses is None:
            return None
        
        responseList = "\n".join("Field: "+ r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in a job application form and the responses to questions about those fields from 
            a candidate's resume. Combine the two into a list of field and succinct, factual answers to fill in those fields. Do not
            output 'Field' and 'Response' but fill in the values of them in the format Field: Response.
            <responses>
            {responseList}
            </responses>
            """
        )

        return StopEvent(result=result)
        

In [ ]:
workflow_form = FormParserRAGWorkflow(timeout=60, verbose=False)
result = await workflow_form.run(
    resume_file="data/resume.pdf",
    application_form="data/application_form.pdf",
)
print(result)

Started parsing the file under job_id c18b91db-cef8-4720-a8da-f48320ff24ea
Here is the combined list of fields and succinct, factual answers:

1. Email: amanlonare95@gmail.com
2. Phone: +81-70-9018-5707
3. Location: Tokyo, Japan
4. First Name: Not provided
5. Last Name: Not provided
6. Professional Experience:
   - Research Software Engineering: Developed a tool for CQRS and Event Sourcing design patterns, designed evaluation of Core Banking application, reduced development time by 15%, facilitated event-driven architecture, and proposed tool integration.
   - Project Research Associate: Developed models for detecting onion infection, formulated a nationwide course on Digital Technology for Smart Agriculture, and achieved high accuracy in detecting onions.
   - Software Developer Intern: Developed functional prototypes for Android application, enabled data visualization using Google Analytics.
7. Projects:
   - Web-based Decision Support System: Developed a web portal for crop monitori